<a href="https://colab.research.google.com/github/nnilayy/Unet/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np
import cv2 
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir "/content/drive/MyDrive/dataset"

In [ ]:
!unzip -q /content/drive/MyDrive/dataset.zip -d /content/drive/MyDrive/

In [130]:
train=ImageDataGenerator(rescale=1/255,
                         rotation_range=40,
                         brightness_range=(0,5),
                         zoom_range=0.2,
                         shear_range=0.2,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode="nearest",
                         )
valid=ImageDataGenerator(rescale=1/255,
                              rotation_range=40,
                              brightness_range=(0,5),
                              zoom_range=0.2,
                              shear_range=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode="nearest",)

In [ ]:
train_dataset=train.flow_from_directory('/content/drive/MyDrive/dataset_3/train',
                                        target_size=(128,128),
                                        batch_size=3,
                                        class_mode='binary',
                                        color_mode='grayscale',
                                        )
val_dataset=valid.flow_from_directory('/content/drive/MyDrive/dataset_3/validation',
                                        target_size=(128,128),
                                        batch_size=3,
                                        class_mode='binary',
                                        color_mode='grayscale'
                                        )

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, InputLayer, Reshape

model = Sequential()
model.add(InputLayer(input_shape=(128, 128, 1)))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(4, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(4, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))
model.summary()






# ---------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------
# (1)Params: 32,000
# (2)Train_Accuracy: 85.45
# (3)Val_Accuracy: 84.44
# model = Sequential()
# model.add(InputLayer(input_shape=(224, 224,3)))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Flatten())
# model.add(Dense(32, activation="relu"))
# model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3), 
              metrics=["accuracy"])

history=model.fit(train_dataset, 
                  steps_per_epoch=32, 
                  epochs=100, 
                  validation_data=val_dataset)

In [134]:
model.save("/content/model_128.h5")

In [ ]:
file="/content/images.jpg"
image=cv2.imread(file)
image=cv2.resize(image,(224,224))
image=np.expand_dims(image,0)
prediction=int(model.predict(image))

if prediction == 0:
  print('Cracked')

elif prediction==1:
  print('Not Cracked')

In [ ]:
# https://www.youtube.com/watch?v=uqomO_BZ44g

In [110]:
import os
def get_file_size(file_path):
  size=os.path.getsize(file_path)
  return size

In [111]:
def convert_bytes(size,unit=None):
  if unit=="KB":
    return print("File size: "+str(round(size/1024,3))+" Kilobytes")
  elif unit=="MB":
    return print("File size: "+str(round(size/(1024*1024),3))+" Megabytes")
  else:
    return print("File size: "+str(size)+"bytes")

In [157]:
model="/content/model_128.h5"

In [136]:
# Model Size Before Model Quantization
convert_bytes(get_file_size(model),"MB")

File size: 0.065 Megabytes


In [153]:
import glob
import cv2
import numpy as array 
files=glob.glob('/content/drive/MyDrive/dataset_3/train/Not Cracked/*')
images=[]
for file in files:
  image=cv2.imread(file)
  image=image/255
  image=np.array(image,dtype=np.float32)
  images.append(image)

In [ ]:
images

In [155]:
def representative_dataset_gen():
   for i in range(100):
      input_image = tf.cast(images[i], tf.uint8)
      input_image = tf.reshape(input_image, [1,128,128,1])
      yield ([input_image])

In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model(model)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations=[tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.experimental_new_converter = True
tflite_model_name="model.tflite"
tflite_model = converter.convert()
open(tflite_model_name,"wb").write(tflite_model)

In [ ]:
# Model Size After Model Quantization
tflite_model="/content/model.tflite"
convert_bytes(get_file_size(tflite_model),"KB")

In [ ]:
# ()Dataset 
# ()Inferencing Code on Arduino
# ()SD Card model transfer
# ()Library Creation and function deletion

In [116]:
import binascii
def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

In [ ]:
tflite_binary = open('model.tflite', 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
header_file = "const unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
open("model_128.h", "w").write(header_file)
convert_bytes(get_file_size('/content/model_128.h'),"KB")

In [ ]:
# train_data=train_dataset[0]
# X_train=train_data[0]
# y_train=train_data[1]

# test_data=val_dataset[0]
# X_test=test_data[0]
# y_test=test_data[1]

# print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
# # print(y_test)

In [ ]:
for layer in model.layers:
  print(layer.get_config()['name'])

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(500):
    yield([x_train[i].reshape(1, 1)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)